In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import mne
from mne.forward import read_forward_solution
from mne.minimum_norm import (make_inverse_operator, apply_inverse, write_inverse_operator)
from mne.stats import summarize_clusters_stc

import scipy
import pickle

from mayavi import mlab
from IPython.display import Image

from mne.decoding import get_coef

mlab.init_notebook('png')
mne.viz.set_3d_backend('mayavi')

In [ ]:
sensors = 'mag'
tmin, tmax = -0.1, 0.6

mainFolder = "..\Data\\"
print('main folder folder: ', mainFolder)

meg_MainFolder = mainFolder + "MEG_Data\Data="
print('meg main folder folder: ', meg_MainFolder)

dataFolder = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', dataFolder)

source_MainFolder = "..\SourceLocalization\SourceEstimates\Data="
sourceFolder = source_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Source folder: ', sourceFolder)

subjects_dir = '..\SourceLocalization\subjects\\'
print('Subjects directory: ', subjects_dir)

forwardModelsFolder = '..\SourceLocalization\ForwardModels\\'
print('Forward models folder: ', forwardModelsFolder)

spatialFiltersFolder = '..\SourceLocalization\SpatialFilters\\Data='
spatialFiltersFolder = spatialFiltersFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Spatial filters folder: ', spatialFiltersFolder)

statResultsFolder  = '..\SourceLocalization\Results\\Data='
statResultsFolder = statResultsFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Statisctics results folder: ', statResultsFolder)

classifiers_MainFolder = "..\Classifiers\Data="    
clsfFolder = classifiers_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Classifiers folder: ', clsfFolder)
    

# Pick the classifiers based on the their training window
peak_indices = [20, 50]
    
print('Peak indices: ', peak_indices)
    
# Task name for the classifiers
task_name = 'all_predLevel'
print('Task name: ', task_name)

# All subjects
s_id_list_all = np.loadtxt(mainFolder+ids_filename, dtype=str)
s_id_list_all = s_id_list_all.tolist()


participant_names = np.loadtxt(mainFolder+participants_filename, dtype=str)
participant_names = participant_names.tolist()


print('Number of subjects: ', len(participant_names))

if os.path.exists(subjects_dir + '\\' + participant_names[0] + '\surf\\'):  
    print('exists!')


In [ ]:
# Functions for loading/splitting data

def getEpochData(s_id):
    
    if int(s_id) < 23:
        fname = dataFolder+'S'+s_id+'\\'+s_id+'_2_tsss_mc_trans_'+sensors+'_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'
    else: 
        fname = dataFolder+'S'+s_id+'\\block_2_tsss_mc_trans_'+sensors+'_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'


    epochs = mne.read_epochs(fname, verbose='error')
    print(fname + ' loaded!')
    return epochs


def splitData(epochs, events=None):
    #print(epochs.event_id)
    if events == None:
        print('No events are given as parameter!')
    
    else:
        print('Requested events: ', events)
        return_list = epochs[events]
     
    print('Events in return list: ', return_list.event_id)
    
    return return_list


def getClassifierWeghts(filename):

    print('Classifier is loaded from ', filename)
    loaded_model = []
    with open(filename, "rb") as f:
        while True:
            try:
                loaded_model.append(pickle.load(f))
            except EOFError:
                break
        
    return loaded_model


# Inverse modeling with BeamFormer on evoked data

## Beamformer

In [ ]:
# Compute spatial filters for beamformer

def computeSpatialFilter(s_name, s_id, evoked, noise_cov, data_cov, condName):
    print('Beamformer')
    print('Computing spatial filter..')
    
    # Load forward model
    fname_fwd = forwardModelsFolder + '\\fwd_sol_ico5_' + s_name + '.fif'
    fwd = mne.read_forward_solution(fname_fwd, verbose=False)
    
    # Compute spatial filters with evoked data, forward model and covariance matrices
    filters = mne.beamformer.make_lcmv(evoked.info, fwd, data_cov=data_cov, reg=0.05, noise_cov=noise_cov, 
                                       pick_ori='max-power', weight_norm='unit-noise-gain', rank='full', reduce_rank=True)
   
    # Save filters
    filters.save(spatialFiltersFolder + s_id + '_filters-lcmv_' + s_name + '.h5', overwrite=True)
    return filters
    

### Prepare data

#### Morph subject's source estimate to template subject
def morphToCommonSpace(stc, s_name, src_ave, smoothAmount=None):
    print('Computing source morph..')
    # Read the source space we are morphing to
    fsave_vertices = [s['vertno'] for s in src_ave]

    morph = mne.compute_source_morph(src=stc, subject_from=s_name, subject_to='fsaverage', 
                                     spacing=fsave_vertices, subjects_dir=subjects_dir, verbose=False,
                                     smooth=smoothAmount)
    tstep = stc.tstep
    
    print('Morphing data to fsaverage..')
    stc_fsave = morph.apply(stc)
    n_vertices_fsave = morph.morph_mat.shape[0]
    
    
    return stc_fsave, n_vertices_fsave, tstep


# generate the inverse solution for group average
def prepareInverseSolution_group(data, tstep, tmin_tmp=0):
    
    src_ave = mne.read_source_spaces(subjects_dir+'fsaverage\\bem\\fsaverage-ico-5-src.fif')

    fsave_vertices = [s['vertno'] for s in src_ave]        
            
    stc_return = mne.SourceEstimate(data, fsave_vertices, tmin_tmp, tstep, subject='fsaverage')
    
    
    return stc_return
    

### Visualize results

#The STC (Source Time Courses) are defined on a source space formed by 7498 candidate locations
# and for a duration spanning 106 time instants.

#Warning: Slide Type
#!!PQt5 is necessary and also run jupyter nbextension enable mayavi --py on command line
# before running the jupyter notebooks and also latest (6.1.1) version of module called 'traits'.

def showResult(s_id, sourceFolder, stc, condName, minimum, maximum, tmin_tmp=0):
    
    initial_time = tmin_tmp
    hemi_list = ['rh', 'lh']
    for hemi in hemi_list:
        print('Hemi: ', hemi)
        
        kwargs = dict(initial_time=initial_time, surface='inflated', hemi=hemi, subjects_dir=subjects_dir,
                      verbose=True, size=(600, 600), spacing='all', background='w',
                      cortex=(211/256,211/256,211/256))
        
        brain = stc.plot(**kwargs, colormap='Oranges')
        brain.scale_data_colormap(fmin=minimum, fmid=(maximum+minimum)/2, fmax=maximum, transparent=True)
        
   
        brain.show_view('lateral');

        brain.save_image(sourceFolder + s_id + '_' + hemi + '_' + condName + '.png')

        Image(filename = sourceFolder +  s_id + '_' + hemi + '_' + condName + '.png', width=600)


def computeActivationMaps(model_list, epochs, tmin):

    meg_data = epochs.get_data()
    epochs.average().plot()
    print("Meg data shape: ", meg_data.shape)
    
    # get classifier weights
    if len(model_list) > 0:
        model = model_list[0] # if model is loaded, it is stored in a list. Therefore we need to get model from index 0
        
        # Get classifier weights
        coef = get_coef(model, 'coef_', inverse_transform=True)
        
        # Compute mean and std of weights
        coef_mean = np.mean(coef)
        coef_std = np.std(coef)
        
        # Standardize the weights
        coef = (coef-coef_mean)/coef_std
        print('shape of coef: ', coef.shape)
    
    
    # Multiplying classifier weights with covariance of data to compute activation maps
    activations_mat = np.zeros((meg_data.shape[0], meg_data.shape[1], meg_data.shape[2]))
    # ntrials, nchannels, ntimes
    
    for t in range(meg_data.shape[2]):
        epochs_tmp = epochs.copy()
        epochs_tmp.crop(tmin=epochs.times[t], tmax=epochs.times[t])
        cov_tmp = mne.compute_covariance(epochs_tmp, verbose=False)

        activations = np.dot(coef, cov_tmp.data)
        if t == 0:
            print('Shape of activations: ', activations.shape)
        
        for i in range( meg_data.shape[0]):
            activations_mat[i, :,t] = activations.reshape(meg_data.shape[1])
            
        del cov_tmp 
    
    # Simulate epoch object with activation maps
    epoched_sim = mne.EpochsArray(activations_mat, epochs.info, tmin=tmin)

    return epoched_sim

### Apply Beamformer
def applyBeamformer(conditions, s_id_list_all, n_subjects, participant_names,
                    tminData, tmaxData, tminNoise, tmaxNoise, tminEpoch, smoothAmount, task_name):
    
    stc_fsave_all_real, n_times = None,  None


    for s in range(n_subjects): 
        s_id = s_id_list_all[s]
        s_name = participant_names[s]
        print(' ------------- ' + s_name + ' ------------- ')
        epochs = getEpochData(s_id)
        print(epochs.event_id)
        print('epochs shape: ', epochs.get_data().shape)
        
        # check if all conditions exist in the epoch (e.g. omissions_living_nores might not exist!)
        conditions = [c for c in conditions if c in epochs.event_id]
        print('Final conditions: ', conditions)
        splits = epochs[conditions]
        print('Events in splits: ', splits.event_id)
        
                
        # Load classifier weights to compute activation maps
        clsf_model_filename = clsfFolder+'S'+s_id+'\\'+s_id+'_clsf_'+task_name+'_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_sm.sav'
        clsf_model = getClassifierWeghts(clsf_model_filename)

        # compute activation maps and simulate epoch object for source localization
        print('Compute activations')
        epoch_sim = computeActivationMaps(clsf_model, splits, tmin=tminEpoch)

        print('Compute noise covariance')
        noise_cov = mne.compute_covariance(epoch_sim, tmin=tminNoise, tmax=tmaxNoise, 
                                           method=['shrunk', 'empirical'], verbose=False) 
        print('Compute data covariance')
        data_cov = mne.compute_covariance(epoch_sim, tmin=tminData, tmax=tmaxData, 
                                          method=['shrunk', 'empirical'], verbose=False)

        # compute average of epochs
        evoked = epoch_sim.average().crop(tmin=tminData, tmax=tmaxData)
        print('Shape of evoked data: ', evoked._data.shape)

        # computer spatial filters by LCMV
        print('Compute filter: ')
        filters = computeSpatialFilter(s_name, s_id, evoked, noise_cov, data_cov, conditions)
        
        print('Apply beamformer: ')
        stc = mne.beamformer.apply_lcmv(evoked=evoked, filters=filters, max_ori_out='signed')
        n_vertices_sample, n_times = stc.data.shape
       

        if s_id != '16' and s_id != '31' and s_id != '40': # for participants with MR data 
            stc_fsave, n_vertices_fsave, tstep = morphToCommonSpace(stc, s_name, src_ave,
                                                                    smoothAmount=smoothAmount)
        else: # for participants without MR data 
            stc_fsave = stc
            
        # initialize the stc data array when computing the first participant
        if s==0:
            print('n_times: ', n_times)
            stc_fsave_all_real = np.zeros((n_vertices_fsave, n_times, n_subjects),)

        
        stc_fsave_all_real[:,:,s] = np.abs(stc_fsave.data.reshape(n_vertices_fsave, n_times))



    return  stc_fsave_all_real, n_times, tstep
                    
            

In [ ]:
# Read fsaverage
src_ave = mne.read_source_spaces(subjects_dir + 'fsaverage\\bem\\fsaverage-ico-5-src.fif')



#### Choose the conditions for contrasting

conditions = ['living_real_8', 'living_real_9', 'living_real_10', 'object_real_8', 'object_real_9', 'object_real_10']
condition_names =  ["real"]

n_subjects = len(s_id_list_all)
print('number of subjects: ', n_subjects)

print('conditions: \n', conditions)


### Define time limits for inverse solutins

tminData, tmaxData = 0.1, 0.4  #0.1, 0.4 #-0.04, 0
print('tmin for data: ', tminData)
print('tmax for data: ', tmaxData)
tminNoiseCov, tmaxNoiseCov = -0.1, -0.05
print('tmin for noise covariance: ', tminNoiseCov)
print('tmax for noise covariance: ', tmaxNoiseCov)
tminEpoch = -0.1
print('tmin for generated epochs: ', tminEpoch)
smoothAmount = 70
print('Smoothing amount: ', smoothAmount)
inv_sol_method = 'beamformer'
print('Inverse solution method: ', inv_sol_method)

### Use beamformer for computing source estimates

In [ ]:
print('======= Applying Beamformer ========')

stc_real_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'

if os.path.exists(stc_real_filename != True) :  # Compute the source estimate

    stc_fsave_all_real, n_times, tstep = applyBeamformer(conditions, s_id_list_all, 
                                                         n_subjects, participant_names, tminData, tmaxData, 
                                                         tminNoiseCov, tmaxNoiseCov, tminEpoch, smoothAmount, task_name)

    print('stc is saved in ', stc_real_filename)
    np.save(statResultsFolder + stc_real_filename, stc_fsave_all_real)

    # Check if there is 0
    if len(np.where(stc_fsave_all_real == 0)[0]) == 0:
        print('No Zeros!')
    else:
        print('There are zeros at ', np.where(stc_fsave_all_real == 0)[0])

else:
    tstep=0.01
    # time period of interest
    tminData_cls_tmp, tmaxData_cls_tmp = 0.1, 0.4
    stc_real_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData)+'_'+str(tmaxData)+'_sm='+str(smoothAmount)+'.npy'
    print('Source estimate exists!\n Loading from file'+statResultsFolder+stc_real_filename+'...')
    stc_fsave_all_real_clfRange = np.load(stc_real_filename)
    print('shape of real sounds source estimates: ', stc_fsave_all_real_clfRange.shape)
    print('There are zeros at ', np.where(stc_fsave_all_real_clfRange == 0))
    
    # Baseline
    tminData_baseline_tmp, tmaxData_baseline_tmp = -0.04, 0
    stc_baseline_filename = 'stc_fsave_allTogether_real_onActivationMaps_'+inv_sol_method+'_'+str(tminData_baseline_tmp)+'_'+str(tmaxData_baseline_tmp)+'_sm='+str(smoothAmount)+'.npy'
    print('Source estimate exists!\n Loading from file'+stc_baseline_filename+'...')
    print(statResultsFolder+stc_baseline_filename)
    stc_fsave_all_real_baseline = np.load(statResultsFolder+stc_baseline_filename)
    print('shape of real sounds source estimates: ', stc_fsave_all_real_baseline.shape)
    print('There are zeros at ',np.where(stc_fsave_all_real_baseline == 0))


### Check data before doing anything!!

print('Baseline: \n', stc_fsave_all_real_baseline)

print('Clf range: \n', stc_fsave_all_real_clfRange)

### Take mean 
# take mean over participants (dimension 2)
stc_fsave_all_real_clfRange_avg = np.mean(stc_fsave_all_real_clfRange, axis=2)
print('Shape stc_fsave_all_real_clfRange_avg after avg over subjects: ', stc_fsave_all_real_clfRange_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_clfRange_avg= np.mean(stc_fsave_all_real_clfRange_avg, axis=1)
print('Shape stc_fsave_all_real_clfRange_avg after avg across time: ', stc_fsave_all_real_clfRange_avg.shape)
stc_fsave_all_real_clfRange_avg.shape

print('stc_fsave_all_real_clfRange_avg: ', stc_fsave_all_real_clfRange_avg)

# take mean over participants (dimension 2)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline, axis=2)
print('Shape stc_fsave_all_real_baseline_avg after avg over subjects: ', stc_fsave_all_real_baseline_avg.shape)
# take mean over time points (dimension 1)
stc_fsave_all_real_baseline_avg = np.mean(stc_fsave_all_real_baseline_avg, axis=1)
print('Shape stc_fsave_all_real_baseline_avg after avg across time: ', stc_fsave_all_real_baseline_avg.shape)
stc_fsave_all_real_baseline_avg.shape

print('stc_fsave_all_real_baseline_avg: ', stc_fsave_all_real_baseline_avg)

### Generate stc group
stc_cls = prepareInverseSolution_group(stc_fsave_all_real_clfRange_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)
stc_baseline = prepareInverseSolution_group(stc_fsave_all_real_baseline_avg, tstep=tstep,
                                            tmin_tmp=tminData_baseline_tmp)


### Plot

'real_baseline_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount)

showResult('fsaverage', sourceFolder, stc_cls,
           'real_clf_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=1300000, maximum=3000000)

showResult('fsaverage', sourceFolder, stc_baseline, 
           'real_baseline_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount), 
           minimum=99000, maximum=440000)

# Compute relative change from baseline
stc_fsave_all_real_diff_avg = 100*(stc_fsave_all_real_clfRange_avg-stc_fsave_all_real_baseline_avg)/stc_fsave_all_real_baseline_avg
 



#print(np.min(stc_fsave_all_real_diff_avg))

# Threshold for plotting
thresh= 700 #550
print('Shape of data larger than threshold: ', np.where(stc_fsave_all_real_diff_avg >thresh)[0].shape)
# zero out the values below threshold
for i in range(stc_fsave_all_real_diff_avg.shape[0]):
    if stc_fsave_all_real_diff_avg[i] < thresh:
        stc_fsave_all_real_diff_avg[i] = 0

stc_diff = prepareInverseSolution_group(stc_fsave_all_real_diff_avg, tstep=tstep, tmin_tmp=tminData_cls_tmp)

#print(np.max(stc_diff.data))
# Visualize the thresholded data
showResult('fsaverage', sourceFolder, stc_diff, 
           '2real_change_allTogether_'+inv_sol_method+'_sm='+str(smoothAmount) +'_pthresh=' + str(thresh), 
           minimum=thresh, maximum=1100)

